# Set up

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize


In [94]:

vip = pd.read_csv("vip_customers.txt", header=None)
orders = pd.read_pickle("customer_orders.pkl")



In [144]:
import numpy as np
import pandas as pd
from pandas import json_normalize

class CustomerDataExtractor:
    def __init__(self, orders, vip):
        self.orders = orders
        self.vip = vip
        self.vip_set = set(vip[0].dropna().astype(int).unique())  # precompute for speed

        # Define columns as class attributes for easy reuse
        self.INT_COLUMNS = ['order_id', 'customer_id', 'product_id', 'item_quantity']
        self.DATE_COLUMNS = ['registration_date', 'order_date']
        self.STR_COLUMNS = ['customer_name', 'product_name', 'category']
        self.ID_COLUMNS = ['customer_id', 'order_id', 'product_id']
        # Category mapping dictionary
        self.category_map = {
            1: 'Electronics',
            2: 'Apparel',
            3: 'Books',
            4: 'Home Goods'
        }

    def clean_quantity(self, series: pd.Series) -> pd.Series:
        series_clean = series.astype(str).str.strip().str.upper()
        series_clean = series_clean.replace('FREE', '0')
        return pd.to_numeric(series_clean, errors='coerce').fillna(0).astype(int)

    def clean_order_id(self, series: pd.Series) -> pd.Series:
        series_str = series.astype(str).replace(['None', 'nan', 'NaN'], pd.NA)
        extracted = series_str.str.extract(r'(\d+)')[0]
        return extracted.astype('Int64')

    def clean_unit_price(self, series: pd.Series) -> pd.Series:
        invalid_vals = {'INVALID', 'invalid', 'N/A', 'NA', ''}
        series_clean = series.astype(str).replace(invalid_vals, np.nan)
        series_clean = series_clean.str.replace(r'[$,]', '', regex=True)
        return pd.to_numeric(series_clean, errors='coerce')


    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        # Clean order_id
        df['order_id'] = self.clean_order_id(df['order_id'])

        # Drop rows with missing critical IDs
        df = df.dropna(subset=self.ID_COLUMNS).reset_index(drop=True)

        # Clean unit_price
        df['unit_price'] = self.clean_unit_price(df['unit_price'])

        # Clean item_quantity
        df['item_quantity'] = self.clean_quantity(df['item_quantity'])

        # Convert ID columns to nullable int
        df[self.ID_COLUMNS] = df[self.ID_COLUMNS].astype('Int64')

        # Convert date columns to datetime with inference and error coercion
        df[self.DATE_COLUMNS] = df[self.DATE_COLUMNS].apply(
            lambda col: pd.to_datetime(col, errors='coerce', infer_datetime_format=True)
        )

        # Ensure string columns are of string dtype
        df[self.STR_COLUMNS] = df[self.STR_COLUMNS].astype(str)
        df['is_vip'] = [True if customer_id in self.vip_set else False for customer_id in df['customer_id']]

        # Add is_vip column
        #df['is_vip'] = df['customer_id'].apply(self.is_vip)

        # Calculate total_item_price and total_order_value_percentage
        df['total_item_price'] = df['unit_price'] * df['item_quantity']
        order_totals = df.groupby('order_id')['total_item_price'].transform('sum')
        df['total_order_value_percentage'] = df['total_item_price'] / order_totals

        # Map categories and fill unmapped with 'Misc'
        df['category'] = df['category'].map(self.category_map).fillna('Misc')

        # Sort the dataframe
        df.sort_values(by=['customer_id', 'order_id', 'product_id'], inplace=True)
        return df

    def extract(self) -> pd.DataFrame:
        # Normalize orders and items JSON data
        cleaned_data = [
            {**customer, 'orders': customer.get('orders', [])}
            for customer in self.orders
        ]

        for customer in cleaned_data:
            customer['orders'] = [
                {**order, 'items': order.get('items', [])}
                for order in customer['orders']
            ]

        orders_df = json_normalize(
            cleaned_data,
            record_path='orders',
            meta=['id', 'name', 'registration_date']
        )

        items_df = json_normalize(
            cleaned_data,
            record_path=['orders', 'items'],
            meta=[['id'], ['orders', 'order_id']],
            record_prefix='item_',
            meta_prefix=''
        )

        # Rename columns for clarity and consistency
        items_df.rename(columns={
            'id': 'customer_id',
            'orders.order_id': 'order_id',
            'item_item_id': 'product_id',
            'item_product_name': 'product_name',
            'item_category': 'category',
            'item_price': 'unit_price',
            'item_quantity': 'item_quantity'
        }, inplace=True)

        orders_df.rename(columns={
            'id': 'customer_id',
            'name': 'customer_name'
        }, inplace=True)

        # Drop unneeded columns
        orders_df.drop(columns=['items', 'shipping_address'], errors='ignore', inplace=True)
        print(orders_df.columns)
        print(items_df.columns)
        
        # Merge orders and items on customer_id and order_id
        data = pd.merge(orders_df, items_df, on=['customer_id', 'order_id'], how='outer')

        # Clean and finalize data
        data = self.clean_data(data)

        return data


In [146]:
extractor = CustomerDataExtractor(orders, vip)
cleaned_df = extractor.extract()
print(cleaned_df.head())

# Export to CSV
cleaned_df.to_csv('cleaned_customer_data.csv', index=False)


Index(['order_id', 'order_date', 'order_total_value', 'customer_id',
       'customer_name', 'registration_date'],
      dtype='object')
Index(['product_id', 'product_name', 'category', 'unit_price', 'item_quantity',
       'customer_id', 'order_id'],
      dtype='object')
   order_id          order_date  order_total_value  customer_id customer_name  \
0         3 2023-09-06 00:42:50           0.000000            1    Customer 1   
1         5 2024-08-28 19:37:56         -57.165786            2    Customer 2   
2         5 2024-08-28 19:37:56         -57.165786            2    Customer 2   
3         5 2024-08-28 19:37:56         -57.165786            2    Customer 2   
4         5 2024-08-28 19:37:56         -57.165786            2    Customer 2   

    registration_date  product_id        product_name category  unit_price  \
0 2022-12-31 04:19:19           1  Item 1 for Order 3     Misc      377.96   
1 2022-05-27 00:23:28           1  Item 1 for Order 5     Misc      342.68   
2 202

/var/folders/lx/8785bxd97hl0x_cz6tbcltlr0000gn/T/ipykernel_6632/748295712.py:59: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  lambda col: pd.to_datetime(col, errors='coerce', infer_datetime_format=True)
/var/folders/lx/8785bxd97hl0x_cz6tbcltlr0000gn/T/ipykernel_6632/748295712.py:59: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  lambda col: pd.to_datetime(col, errors='coerce', infer_datetime_format=True)
